# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [6]:
#Libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, classification_report, confusion_matrix, accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [8]:
#your code here
spaceship_df = spaceship.dropna()

spaceship_df.columns = spaceship_df.columns.str.lower().str.replace('[^0-9a-zA-Z]+','_',regex=True)
spaceship_df

,passengerid,homeplanet,cryosleep,cabin,destination,age,vip,roomservice,foodcourt,shoppingmall,spa,vrdeck,name,transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [10]:

spaceship_df[['cryosleep','vip','transported']] =  spaceship_df[['cryosleep','vip','transported']].replace({True: 1, False: 0})

spaceship_df['cabin']=spaceship_df['cabin'].str.split('/').str[0]
spaceship_use = spaceship_df.drop(['passengerid','name'], axis=1)

C:\Users\LAURA\AppData\Local\Temp\ipykernel_16636\4185191010.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spaceship_df[['cryosleep','vip','transported']] =  spaceship_df[['cryosleep','vip','transported']].replace({True: 1, False: 0})
C:\Users\LAURA\AppData\Local\Temp\ipykernel_16636\4185191010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship_df[['cryosleep','vip','transported']] =  spaceship_df[['cryosleep','vip','transported']].replace({True: 1, False: 0})
C:\Users\LAURA\AppData\Local\Temp\ipykerne

In [13]:

spaceship_cate_trans = pd.get_dummies(spaceship_use[['homeplanet','destination','cabin']],drop_first=True)

spaceship_use = pd.concat([spaceship_use, spaceship_cate_trans], axis=1)
spaceship_use = spaceship_use.drop(columns=['homeplanet','destination','cabin'])
spaceship_use

,cryosleep,age,vip,roomservice,foodcourt,shoppingmall,spa,vrdeck,transported,homeplanet_Europa,homeplanet_Mars,destination_PSO J318.5-22,destination_TRAPPIST-1e,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_T
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,True,False,False,True,True,False,False,False,False,False,False
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,False,False,False,True,False,False,False,False,True,False,False
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,True,False,False,True,False,False,False,False,False,False,False
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,True,False,False,True,False,False,False,False,False,False,False
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,False,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0,True,False,False,False,False,False,False,False,False,False,False
8689,1,18.0,0,0.0,0.0,0.0,0.0,0.0,0,False,False,True,False,False,False,False,False,False,True,False
8690,0,26.0,0,0.0,0.0,1872.0,1.0,0.0,1,False,False,False,True,False,False,False,False,False,True,False
8691,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0,True,False,False,False,False,False,False,True,False,False,False


In [15]:
features = spaceship_use.drop(columns=['transported'])
target = spaceship_use['transported']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

normalizer = MinMaxScaler()
normalizer.fit(X_train)

X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)
X_train_norm =pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm =pd.DataFrame(X_test_norm, columns = X_test.columns)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [23]:
#your code here
forest_class = RandomForestClassifier(n_estimators=100,max_depth=20)
forest_class.fit(X_train_norm, y_train)
pred_forest = forest_class.predict(X_test_norm)

- Evaluate your model

In [25]:
#your code here
print(classification_report(y_test, pred_forest))

              precision    recall  f1-score   support

           0       0.78      0.79      0.79       661
           1       0.79      0.78      0.79       661

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322



**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [27]:
#your code here

grid = {"n_estimators": [50, 100, 200, 500],
        "max_leaf_nodes": [250, 500, 1000, None],
        "max_depth": [10, 30, 50]}

- Run Grid Search

In [29]:
model = GridSearchCV(estimator = forest_class, param_grid = grid, cv=5)
model.fit(X_train_norm, y_train)


GridSearchCV(cv=5, estimator=RandomForestClassifier(max_depth=20),
             param_grid={'max_depth': [10, 30, 50],
                         'max_leaf_nodes': [250, 500, 1000, None],
                         'n_estimators': [50, 100, 200, 500]})

- Evaluate your model

In [31]:
forest_class2 = RandomForestClassifier(n_estimators=500,max_depth=10, max_leaf_nodes=None)
forest_class2.fit(X_train_norm, y_train)
pred_forest2 = forest_class2.predict(X_test_norm)

In [32]:
print(classification_report(y_test, pred_forest))

              precision    recall  f1-score   support

           0       0.78      0.79      0.79       661
           1       0.79      0.78      0.79       661

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322



In [37]:
cv_results = model.cv_results_
mean_val_scores = cv_results['mean_test_score']
mean_val_scores

array([0.80488124, 0.80317777, 0.80469328, 0.80715432, 0.80412474,
       0.80431539, 0.80582839, 0.80507117, 0.80563971, 0.80431377,
       0.80469238, 0.80526074, 0.80015159, 0.80450281, 0.8060176 ,
       0.80677518, 0.79996166, 0.79958305, 0.80128634, 0.80109748,
       0.78746972, 0.79163441, 0.79295946, 0.79239109, 0.78406314,
       0.78444157, 0.78633425, 0.78671286, 0.78595546, 0.78671322,
       0.78633533, 0.78614468, 0.79712254, 0.80052912, 0.80015105,
       0.80223385, 0.79030848, 0.78936258, 0.79106552, 0.79182291,
       0.78482053, 0.78444228, 0.7869019 , 0.78406296, 0.78103606,
       0.78746954, 0.7853871 , 0.78690225])